# Face Recognition with a Support Vector Machine

Qin Zhao PhD & Bas Michielsen MSc

Face recognition is the ability for the computer to identify a person based on an image of their face. This is a classification problem, where each possible person is a class, and the provided image should lead to 1 specific class with a as high as feasible certainty. In order to train a classification model with this, we need as many as possible images of the same person's face. In this exercise we use a dataset that is provided by sklearn called the 'Olivetti Faces'. Mote information can be found at https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_olivetti_faces.html

The dataset has 400 images for 40 different persons, namely exactly 10 images per person. The dataset is also labelled, so for every image we know the correct class. Now we can use this dataset to train a model.

We start by importing scikit-learn, numpy, matplotlib and seaborn the Python libraries we will be using for this analysis. 

First, we show the versions of these libraries (that is always wise to do in case you have to report problems running the notebooks!) and use the inline plotting mode statement.

In [ ]:
import sklearn as sk
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

print('scikit-learn version:', sk.__version__)
print('numpy version:', np.__version__)
print('matplotlib version:', matplotlib.__version__)
print('seaborn version:', sns.__version__)

Then let us load the data.

In [ ]:
from sklearn.datasets import fetch_olivetti_faces
faces = fetch_olivetti_faces()

To get an impression of the data, here we show the first photo of each of the 40 unique people in the dataset.

In [ ]:
fig, subplots = plt.subplots(nrows=4, ncols=10, figsize=(18, 9))
subplots = subplots.flatten()
    
for id in np.unique(faces.target):
    index = id *10 # Because there are 10 images per person.
    subplots[id].imshow(faces.images[index], cmap="gray")
    subplots[id].set_xticks([])
    subplots[id].set_yticks([])
    subplots[id].set_title("id: {}".format(id))
plt

## Preprocessing
Given that we are using images, there is no such thing as feature selection because you cannot select some pixels to be better indicators than other pixels. We therefore have little to do in terms of preprocessing other than splitting the dataset into a trainset and testset.
We select a stratisfied test size of .3, which means that from every person 30% of the images will be used as testset. Since every person has 10 images, 7 will be selected for training and 3 will be selected for testing. 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(faces.data, faces.target, test_size=.3, random_state=0, stratify=faces.target)

## Modelling
In this step we are going to use the trainset only to fit the model. In this case a Support Vector Machine for classification.

In [ ]:
from sklearn.svm import SVC # SVC is for classification, an SVR for regression exists as well.
model = SVC()
model.fit(X_train, y_train)

After that we can let the model do predictions on the testset and see the accuracy of those predictions.

In [ ]:
pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
acc = accuracy_score(pred, y_test)
print(acc)

An accuracy of a little more than 88% is not too bad. In fact this is quite good. 😊

## Evaluation
Now let us see if we can shed some light on the results. The first thing we can do here is print a classification report. This shows for every one of the classes how well the model performed. There are 2 metrics to consider, namely Precision and Recall. The F1-score is a quality score based on precision and recall together.

Precision: When the model predicts class X, how often was it correct?

Recall: When the correct class is X, how often did the model predict it so?

Note that both Precision and Recall are fractions, so 1.00 is 100% of the times, and 0.00 is 0% of the times.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

It appears that class 39 has a fairly low Precision, this means that often the model says 39 when in fact it is another person. In other words, quite a number of people get identified as person 39 incorrectly. Also it appears that the classes 0, 25 and 34 have a low Recall, this means that these people often get identified as some else.

The next thing we can do here is making a confusion matrix. This matrix shows all 40 ids (0 - 39) as rows and on every row shows the made predictions for that id in the testset. If the prediction is correct if will show in the [id, id] cell, if not, it will show in another cell and we can see the id that the model predicted instead. Since the testset is stratisfied at 3 images per person, 3 is the maximum of correct classifications, so that should be the vmax parameter.

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, pred)
plt.figure(figsize=(15, 15)) # A size of 15 because it feels good! You can try other numbers if you want.
sns.heatmap(matrix, annot=True, cbar=None, vmax=3)

Of course here we see a similar result as the classification report. In the vast majority of cases the model identifies all 3 images of the same person correctly. However, the column 39 shows many 1s for other rows than 39, and the rows 0, 25 and 34 show many 1s in columns other than the correct one.

# SECTION 1

## Cross Validation

- I use cross_val_score because it automatically performs k-fold cross validation
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
- I set cv=5 because the exercise asks for 5-fold cross validation
- I apply this on X_train and y_train because I want to validate the training process

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cv_scores = cross_val_score(model, X_train, y_train, cv=5)
# I print each score because the exercise asks to show what the accuracy scores are
print("Cross-validation scores:", cv_scores)
# I calculate the mean because the exercise asks for the average accuracy score
print("Average accuracy:", cv_scores.mean())

# SECTION 2

## StratifiedKFold

- I import StratifiedKFold because the exercise specifically asks to use this function
- Reference: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
- I use n_splits=5 because the exercise asks for k=5
- I set shuffle=True and random_state=0 to ensure reproducible results

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
accuracies = []

# I loop through the folds because I need to manually train and test on each fold
for fold, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
    # I split the data using the indices because StratifiedKFold gives me train/val indices
    X_fold_train = X_train[train_index]
    X_fold_val = X_train[val_index]
    y_fold_train = y_train[train_index]
    y_fold_val = y_train[val_index]
    
    # I create a new model for each fold because each fold needs its own trained model
    fold_model = SVC()
    fold_model.fit(X_fold_train, y_fold_train)
    
    # I predict on the validation set because I need to evaluate this fold's performance
    fold_pred = fold_model.predict(X_fold_val)
    # I calculate accuracy because the exercise asks for accuracy score of each fold
    fold_acc = accuracy_score(y_fold_val, fold_pred)
    # I append to the list because I need to calculate the average later
    accuracies.append(fold_acc)
    
    # I print each fold's accuracy because I need to print out the accuracy score of each fold
    print("Fold", fold+1, "accuracy:", fold_acc)

# I calculate the mean because I need to calculate the average accuracy score
print("Average accuracy:", np.mean(accuracies))

# SECTION 3

## Glasses Classification

- I will solve a binary classification problem because the exercise asks to detect glasses instead of recognizing people
- Reference for binary classification with SVM: https://scikit-learn.org/stable/modules/svm.html#classification


In [ ]:
# I create the glasses_ranges list
glasses_ranges = [
    (10, 19), (30, 32), (37, 38), (50, 59), (63, 64),
    (69, 69), (120, 121), (124, 129), (130, 139), (160, 161),
    (164, 169), (180, 182), (185, 185), (189, 189), (190, 192),
    (194, 194), (196, 199), (260, 269), (270, 279), (300, 309),
    (330, 339), (358, 359), (360, 369)
]

# I create a target array of all zeros because initially no one has glasses (0 = no glasses)
glasses = np.zeros(len(faces.data))

# I loop through the ranges because I need to mark all images with glasses as 1
for start, end in glasses_ranges:
    # I set indices from start to end+1 because Python ranges are exclusive at the end
    # I use end+1 because the exercise says "up to and including" the end index
    glasses[start:end+1] = 1

# I print these statistics because I want to verify the target variable was created correctly
print("Total images with glasses:", np.sum(glasses))
print("Total images without glasses:", len(glasses) - np.sum(glasses))

# I split the data because the exercise asks to create a training & test set for this new problem
# I use stratify=glasses because I want both sets to have similar proportions of glasses/no glasses
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(
    faces.data, glasses, test_size=.3, random_state=0, stratify=glasses
)

# I train a new SVM model I need to train support vector machine for the new target
model_glasses = SVC()
model_glasses.fit(X_train_g, y_train_g)

# I predict on the test set because I need to evaluate the model's performance
pred_glasses = model_glasses.predict(X_test_g)
acc_glasses = accuracy_score(pred_glasses, y_test_g)
print("Glasses classification accuracy:", acc_glasses)

# I show the classification report because the exercise asks to show a classification report
print(classification_report(y_test_g, pred_glasses, target_names=['No Glasses', 'Glasses']))

# I apply cross validation because the exercise asks to apply cross validation to avoid overfitting
cv_scores_glasses = cross_val_score(model_glasses, X_train_g, y_train_g, cv=5)
print("Cross-validation scores:", cv_scores_glasses)
print("Average accuracy:", cv_scores_glasses.mean())